In [1]:
import pysam
import collections
import random
import pandas as pd
import numpy as np
%load_ext rpy2.ipython


/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: During startup - 
  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: Warning messages:

  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: 1: package ‘methods’ was built under R version 3.2.4 

  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: 2: package ‘datasets’ was built under R version 3.2.4 

  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarning: 3: package ‘utils’ was built under R version 3.2.4 

  rpy2.rinterface.initr()
/shared/conda-install/envs/cgat-devel-lite/lib/python2.7/site-packages/rpy2/robjects/robject.py:6: UserWarnin

In [2]:
infile = "mapping.dir/Nxf1-GFP-R1.bam"

This notebook contains test code to examine whether indel mutations in UMIs present a substainal effect. To test this, we first recognise that a one base pair indel will suck a base of the actual genomic sequencing into the UMI, shifting the mapping position of the by one base, and including a genome determined base as the final base of the UMI.

To test this we will parse the bam file and fill a dictionary: key="[contig][position]", value= counter of UMIs for each position. Then we'll parse the dictionary keys and look to see whether the positions +1 exist in the dictionary keys. If they do, we'll compare the umi profiles between the postions. 

First we will filter for UMIs that have the genomic base in the final UMI position. Then we will check for each possible one bp deletion in the reference UMI and see if that UMI exists in the +1 position. We will count the number of UMIs in the + 1 position that match this criteria. This way we will count the number of UMIs that may have arisen from a deletion. 

This will then be compared to a random sample

A function to calculate the number of the reference and plus 1 base UMIs that are part of deletion pairs.

In [222]:
def getDelNumber(counter1, counter2, genomic_base):
    
    umis1 = counter1.keys()
    umis2 = counter2.keys()
    
    found1 = set()
    found2 = set()
    
    filtered_set2 = set([umi[:-1] for umi in umis2 if umi[-1] == genomic_base])
    for umi in umis1:
        for i in range(len(umi)):
            del_umi = umi[:i] + umi[i+1:]
            if del_umi in filtered_set2:
                found1.add(umi)
                found2.add(del_umi+genomic_base)
            
    return  len(found2)


now a function to parse a bam and create a dictionary containing the count of each UMI at each position.

Because whether we care about +1 or -1 and whether we want to rev comp the sequence or not, we will deal with plus and minus strands seperately. 

In [4]:
def parse_samfile(infile):
    '''Parses a bamfile and returns three dictionaries, the first is a dictionary of counters
    with the count of each umi at each bases on each contig, the second is the first bases matched
    reads at that position, and the third is the distribution of the UMIs in the file'''
    insam = pysam.Samfile(infile, "rb")

    umi_pos = collections.defaultdict(lambda:collections.defaultdict(lambda: collections.Counter()))
    umi_dist=collections.Counter()
    genomic_bases = collections.defaultdict(lambda: collections.defaultdict(str))
    inreads = insam.fetch()
    for read in inreads:
        if read.is_unmapped:
            continue

        if read.mate_is_unmapped and paired:
            continue

        if read.is_read2:
            continue

        is_spliced = False

        if read.is_reverse:
            continue
        
        else:
            pos = read.pos
            if read.cigar[0][0] == 4:
                pos = pos - read.cigar[0][1]
            start = pos

            if ('N' in read.cigarstring or
                (read.cigar[-1][0] == 4 and
                 read.cigar[-1][1] > soft_clip_threshold)):
                is_spliced = True

        umi = read.qname.split("_")[-1]
        chrom = insam.get_reference_name(read.tid)
        umi_pos[chrom][pos][umi] += 1
        umi_dist[umi] += 1
        
        if read.cigar[0][0] == 0:
            genomic_bases[chrom][pos] = read.query_sequence[0]
                
    return umi_pos, umi_dist, genomic_bases

Start with a single sample as test. 

In [46]:
undeduped_umi_pos, undeduped_umi_dist, undeduped_bases = parse_samfile(infile)

Now we need to go through each pair of adjecent positions and calculate the fraction of the UMIs at position +1 that could be explained as deletions of UMIs at the reference position. Then randomise the UMIs at the +1 position and do the same. 

In [6]:
def randomise_position(umi_counter, umi_dist):
    '''Takes a counter of UMI frequencies and create a randomised distribution by replacing the UMIs 
    in the input which UMIs sampled from the genomewide distribution'''
    
    return {umi: count for umi, count in 
            zip(np.random.choice(umi_dist["index"], 
                                 size=len(umi_counter.keys()),
                                 replace=False,
                                 p=umi_dist["freq"]),
                                 umi_counter.values())}

In [122]:
def calculate_deletion_rate(umi_pos, umi_dist, genomic_bases, sum_func=getDelNumber):
    '''Find positions where position+1 also has UMIs and calculate the deletion rate,
    in comparision to randomised UMIs'''

    # convert count umi_dist list to dataframe mapping umi to frequencies
    umi_dist = pd.Series(umi_dist, name="count").reset_index()
    umi_dist["freq"] = umi_dist["count"]/umi_dist["count"].sum()
    
    results_accumulator = []
    random_accumulator = []
    for contig in umi_pos:
        for position in umi_pos[contig]:

            genomic_base = genomic_bases[contig][position]
            if genomic_base == str():
                continue
            this =  umi_pos[contig][position]
            if position +1 in umi_pos[contig]:
                other = umi_pos[contig][position + 1]
            else:
                other = collections.Counter()
                
            rand_other = randomise_position(other, umi_dist)
            results_accumulator.append(sum_func(this, other, genomic_base))
            random_accumulator.append(sum_func(this, rand_other, genomic_base))

    results_frame = pd.DataFrame({"deletion_rate": results_accumulator})
    random_accumulator = pd.DataFrame({"deletion_rate": random_accumulator})

    results_frame["random"] = False
    random_accumulator["random"] = True

    results_frame = pd.concat([results_frame, random_accumulator])
    return results_frame



Now we are ready to run this on our selected test case:

In [201]:
undedup_deletion_numbers = calculate_deletion_rate(undeduped_umi_pos, undeduped_umi_dist, undeduped_bases, getDelNumber)

What we really want to look at is not the number of UMIs that might result from Indels, but the fractions, so we need to divide by the total number of UMIs seen

In [202]:
undedup_deletion_numbers.groupby("random").sum()/sum([len(undeduped_umi_pos[contig][base].keys()) for contig in undeduped_umi_pos for base in undeduped_umi_pos[contig]])

,deletion_rate
random,
False,0.113264
True,0.105064


First things to note is that 10% is quite a lot. Second is that it is around 10% in both the actaul data and in the randomised control. This is not so unexpected because there are 5 possible deletion matches for each UMI, and if there are multiple UMIs at each of the adjecent positions, then it there is quite a high chance one will match. Particularly is you consider that many of the UMIs will occur in positions with many UMIs at them. These are more likely to have adjecent positions with reads mapping, and those positions are also likely to have many UMIs mapping. 

The total enrichment is lower 11.3%/10% ~ 1.08 fold enrichement. 

Now  to run on all of the files in repilcate 1 of the experiment. Frist find the deduplicated files:

In [73]:
import glob
import os
infiles = pd.Series(glob.glob("mapping.dir/*R1.bam"))
infiles.index = infiles.apply(os.path.basename)
infiles

SRSF5-GFP-R1.bam      /ifs/projects/ians/umisdeduping/iCLIP_deduping...
SRSF7-GFP-R1.bam      /ifs/projects/ians/umisdeduping/iCLIP_deduping...
Control-GFP-R1.bam    /ifs/projects/ians/umisdeduping/iCLIP_deduping...
SRSF3-GFP-R1.bam      /ifs/projects/ians/umisdeduping/iCLIP_deduping...
SRSF6-GFP-R1.bam      /ifs/projects/ians/umisdeduping/iCLIP_deduping...
SRSF4-GFP-R1.bam      /ifs/projects/ians/umisdeduping/iCLIP_deduping...
Nxf1-GFP-R1.bam       /ifs/projects/ians/umisdeduping/iCLIP_deduping...
SRSF1-GFP-R1.bam      /ifs/projects/ians/umisdeduping/iCLIP_deduping...
SRSF2-GFP-R1.bam      /ifs/projects/ians/umisdeduping/iCLIP_deduping...
dtype: object

Now define a wrapper function that encloses the whole analysis in one function:

In [203]:
def run_number_analysis(infile):
    print "analysing ", infile
    umi_pos, umi_dist, bases = parse_samfile(infile)
    deletion_rates = calculate_deletion_rate(umi_pos, umi_dist, bases, getDelNumber)
    total = sum([len(umi_pos[contig][base].keys()) for contig in umi_pos for base in umi_pos[contig]])
    return (deletion_rates.groupby("random").sum()/total)["deletion_rate"]

and apply this accross all the input files:

In [204]:
results = infiles.apply(run_number_analysis)

analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/SRSF5-GFP-R1.bam
analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/SRSF7-GFP-R1.bam
analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/Control-GFP-R1.bam
analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/SRSF3-GFP-R1.bam
analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/SRSF6-GFP-R1.bam
analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/SRSF4-GFP-R1.bam
analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/Nxf1-GFP-R1.bam
analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/SRSF1-GFP-R1.bam
analysing  /ifs/projects/ians/umisdeduping/iCLIP_deduping/SR_iCLIP_test3/mapping.dir/SRSF2-GFP-R1.bam


Calulate the enrichment:

In [227]:
results["enrichment"] = results[False]/results[True]
results

random,False,True,enrichment
SRSF5-GFP-R1.bam,0.085877,0.084387,1.017658
SRSF7-GFP-R1.bam,0.039137,0.036899,1.060653
Control-GFP-R1.bam,0.022180,0.016067,1.380488
SRSF3-GFP-R1.bam,0.048327,0.043004,1.123770
SRSF6-GFP-R1.bam,0.056353,0.053947,1.044607
SRSF4-GFP-R1.bam,0.101630,0.094871,1.071246
Nxf1-GFP-R1.bam,0.113264,0.104850,1.080242
SRSF1-GFP-R1.bam,0.072417,0.064592,1.121152
SRSF2-GFP-R1.bam,0.064955,0.063312,1.025946


Some stats:

In [232]:
results.apply(np.log).describe()

,False,True,enrichment
count,9.000000,9.000000,9.000000
mean,-2.803755,-2.897659,0.093904
std,0.509219,0.578451,0.092416
min,-3.808545,-4.130982,0.017504
25%,-3.029771,-3.146459,0.043641
50%,-2.734061,-2.759676,0.068822
75%,-2.454841,-2.472345,0.114357
max,-2.178036,-2.255221,0.322437


So the mean enrichment is 1.1x (+/- standard deviation of 0.1) over random. Pretty low. It could be argued that even if its low we could ebefit from removing it. Applying the same rules as directional, we will see how many are within the 2n+1 threshold:

In [ ]:
def getThresholdedNumber(counter1, counter2, genomic_base):

    umis1 = counter1.keys()
    umis2 = counter2.keys()
    
    found1 = set()
    found2 = set()
    
    filtered_set2 = set([umi[:-1] for umi in umis2 if umi[-1] == genomic_base])
    for umi in umis1:
        for i in range(len(umi)):
            del_umi = umi[:i] + umi[i+1:]
            if del_umi in filtered_set2 and counter1[umi] > 2*counter2[del_umi+genomic_base] + 1:
                found1.add(umi)
                found2.add(del_umi+genomic_base)
            
    return  len(found2)

In [223]:
undedup_threshold_numbers = calculate_deletion_rate(undeduped_umi_pos, undeduped_umi_dist, undeduped_bases, getThresholdedNumber)

In [225]:
undedup_threshold_numbers.groupby("random").sum()/sum([len(undeduped_umi_pos[contig][base].keys()) for contig in undeduped_umi_pos for base in undeduped_umi_pos[contig]])

,deletion_rate
random,
False,0.041546
True,0.040011


So of those 11% we found earlier that could be indel related purely on the basis of sequence, 4% also fulfill the count threshold. However, just as many of the random ones do as well. So if we were to remove these, of the 4.2% we would remove 4% would be erroneously removed. 

What are these enrichments compared to the enrichments caused by edit_disatance enrichments?

In [156]:
edit_distance = pd.read_csv(
    "dedup_directional.dir/Nxf1-GFP-R1_edit_distance.tsv", sep="\t")
edit_distance = edit_distance.set_index("edit_distance")
edit_distance


,directional-adjacency,directional-adjacency_null,unique,unique_null
edit_distance,,,,
Single_UMI,1258729,1258729,1241294,1241294
0,0,54,0,63
1,219,654,17526,924
2,5716,3950,5922,5534
3,13516,13840,14210,18111
4,67613,68440,67835,76169
5,14082,14208,13088,17780
6,0,0,0,0


In [77]:
edit_distance_fractions = edit_distance.drop("Single_UMI", axis=0).apply(lambda x: x/sum(x))

In [78]:
edit_distance_fractions["unique"]/edit_distance_fractions["unique_null"]

edit_distance
0     0.000000
1    24.984797
2     1.008260
3     0.703621
4     0.867225
5     0.660211
6          NaN
dtype: float64

So the substitution errors are a 25x enrichment, rather than a 1.1 fold enrichment for the deletions. 